# IBKR Tax

In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
from currency_converter import CurrencyConverter
c = CurrencyConverter(fallback_on_missing_rate=True)

# Utils

In [ ]:
def addSumCol(df):
    df.loc[:,'Sum'] = df.sum(numeric_only=True, axis=1)
    return df

In [ ]:
def addSumRow(df):
    if isinstance(df.index, pd.MultiIndex):
        myRange = range(df.index.nlevels)
        myIndex = tuple('Sum' for x in myRange)
        df.loc[myIndex,:]= df.sum(numeric_only=True, axis=0)
    else:
        df.loc['Sum']= df.sum(numeric_only=True, axis=0)
    return df

# Read Data

Requirements:
- English activity statement
- Year of activity statement 2020 and older
- All Options, Futures, CFDs closed before year end
- Only single short put and short call (Line 25 Losses from the disposal of worthless assets as per section 20(1) of the Income Tax Act not implemented)
- Manual credit of Withholding tax
- No classification of REITs as investment fund required

In [ ]:
myfile = "MY_ACTIVITY_STATEMENT.csv"

https://stackoverflow.com/questions/27020216/import-csv-with-different-number-of-columns-per-row-using-pandas/57824142#57824142

In [ ]:
### Loop the data lines
with open(myfile, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
df = pd.read_csv(myfile, header=None, delimiter=",", names=column_names)

# Trades

In [ ]:
df_trades = df[df.iloc[:,0] == "Trades"].dropna(how='all', axis=1)
df_trades, df_trades.columns = df_trades.iloc[1:] , df_trades.iloc[0]
df_trades.columns.name = None

In [ ]:
# obtain asset type
try:
    df_trades[['Asset','Category']] = df_trades["Asset Category"].str.split("-", expand=True).copy()
except:
    df_trades["Asset"] = df_trades["Asset Category"]

# remove subheader
df_trades = df_trades[(df_trades["Realized P/L"]!="Realized P/L") & ~(df_trades["Header"].str.contains("SubTotal|Total"))].copy()

# convert dtypes
df_trades["Realized P/L"] = df_trades["Realized P/L"].astype(float)

# remove empty rows
df_trades = df_trades[df_trades["Realized P/L"].notnull()] #df_trades["Realized P/L"]!=0) &
 
# convert to datetime
df_trades["Date/Time"] = pd.to_datetime(df_trades["Date/Time"],  infer_datetime_format=True)

df_futures = df_trades[df_trades["Asset Category"].str.contains("Futures")] \
    if df_trades["Asset Category"].str.contains("Futures").sum() > 0 else pd.DataFrame()
df_options = df_trades[df_trades["Asset Category"].str.contains("Options")] \
    if df_trades["Asset Category"].str.contains("Options").sum() > 0 else pd.DataFrame()
df_cfd = df_trades[df_trades["Asset Category"].str.contains("CFD")] \
    if df_trades["Asset Category"].str.contains("CFD").sum() > 0 else pd.DataFrame()

df_trades = df_trades.reset_index(drop=True)

In [ ]:
# convert to EUR using ECB rates
df_trades["P/L [€]"] = df_trades.apply(lambda row: c.convert(
    row["Realized P/L"] , row.Currency, date=row["Date/Time"]), axis=1)
df_trades["P [€]"] = df_trades["P/L [€]"].apply(lambda row: row if row > 0 else 0)
df_trades["L [€]"] = df_trades["P/L [€]"].apply(lambda row: row if row < 0 else 0)

In [ ]:
PL_TradesDet = df_trades.groupby(["Currency", "Asset"]).sum()
PL_Trades = df_trades.groupby(["Asset"]).sum()

# Dividend

In [ ]:
def process_df(df):
    # set first row as header
    df, df.columns = df.iloc[1:] , df.iloc[0]
    
    # remove rows with sum
    df = df[~df.Currency.str.contains("Total")]
    
    # conver dtypes
    df["Amount"] = df["Amount"].astype(float)
    
    # convert to datetime
    df["Date"] = pd.to_datetime(df["Date"],  infer_datetime_format=True)
    
    # convert to EUR using ECB rates
    df["Amount [€]"] = df.apply(lambda row: c.convert(row["Amount"] , row["Currency"], date=row["Date"]), axis=1)
    
    # label CFD dividens
    df["Description"] = df["Description"].str.replace(" ","")
    df[['Symbol','TrashCol']] = df["Description"].str.split("(", n=1, expand=True).copy()
    df[['Country','TrashCol']] = df["TrashCol"].str.split(")", n=1, expand=True).copy()
    df["Country"] = df["Country"].str.extract(r'(^\D+)').fillna("CFD")
    df["Asset"] = "Stocks"
    df.loc[df.Symbol.str.endswith("n"), "Asset"] = "CFDs" 
    
    # remove index from column names
    df.columns.name = None
    df.reset_index(drop=True)
    
    return df

In [ ]:
df_div = df[df.iloc[:,0] == "Dividends"].dropna(how='all', axis=1)
df_wtax = df[df.iloc[:,0] == "Withholding Tax"].dropna(how='all', axis=1)
df_871 = df[df.iloc[:,0] == "871(m) Withholding"].dropna(how='all', axis=1)

In [ ]:
df_div = process_df(df_div)
df_wtax = process_df(df_wtax) if not df_wtax.empty else df_wtax

In [ ]:
df_871 = process_df(df_871) if not df_871.empty else df_871
df_871["Asset"] = "CFDs"
df_wtax = df_wtax.append(df_871)

In [ ]:
WithholdingTax = df_wtax.groupby(["Currency", "Asset", "Country"]).sum() if not df_wtax.empty else df_wtax
Dividends = df_div.groupby(["Currency", "Asset", "Country"]).sum()

In [ ]:
# reduce df & convert dtypes
df_interest = df[df.iloc[:,0] == "Interest"].dropna(how='all', axis=1)
df_interest, df_interest.columns = df_interest.iloc[1:] , df_interest.iloc[0]
df_interest = df_interest[~df_interest["Currency"].str.contains("SubTotal|Total")]
df_interest["Amount"] = df_interest["Amount"].astype(float)

# convert to datetime
df_interest["Date"] = pd.to_datetime(df_interest["Date"],  infer_datetime_format=True)
# convert to EUR using ECB rates
df_interest["Amount [€]"] = df_interest.apply(lambda row: c.convert(row["Amount"] , row["Currency"], date=row["Date"]), axis=1)
df_interest.columns.name = None



In [ ]:
# aggregate positive and negative
Interest = df_interest.groupby(["Currency"])[['Amount','Amount [€]']].agg([
    ("Received", lambda x: x[x>0].sum()),
    ("Paid", lambda x: x[x<0].sum())
])

# Futures / Option Premiums

- Only applies if every position is closed and opened during the year (i.e. Options)

In [ ]:
def getTradesPnl(df, multiplier):
    if df.empty:
        return pd.DataFrame(), pd.DataFrame()
    else:
        try:
            df["TPrice"] = df["T. Price"].astype(float) * multiplier
            df["TPrice"] = df["TPrice"] * -df["Quantity"].astype(int)
            df["TPrice"] = df["TPrice"] + df["Comm/Fee"].astype(float) 
            df["Pnl"] = df.apply(lambda row: c.convert(
                        row["TPrice"] , row.Currency, date=row["Date/Time"]), axis=1)
            df["cummulative"] = df["Pnl"].cumsum()

            df["Open"] = df.Code.str.match(r'(C)(?!\w)') # regex should also include (O) not prepended with word char
            df["Open"] = df["Open"].cumsum().shift().fillna(0)

            s1 = df.groupby("Open").last()["Symbol"]
            s2 = df.groupby("Open").sum()["Pnl"]
        except:
            s1 = pd.Series([0], name="Symbol")
            s2 = pd.Series([0], name="Pnl")

        df_resultDet = pd.concat([s1, s2], axis=1)
        df_result = pd.DataFrame()

        try:
            df_result["P [€]"] = [df_resultDet["Pnl"].apply(lambda row: row if row > 0 else 0).sum()]
            df_result["L [€]"] = [df_resultDet["Pnl"].apply(lambda row: row if row < 0 else 0).sum()]
        except:
            pass

        return df_result, df_resultDet
    

In [ ]:
multiplier = 1
Futures, FuturesDet = getTradesPnl(df_futures.copy(), multiplier)
Futures

In [ ]:
multiplier = 100
Options, OptionsDet = getTradesPnl(df_options.copy(), multiplier)
Options

In [ ]:
multiplier = 1
Cfd, CfdDet = getTradesPnl(df_cfd.copy(), multiplier)
Cfd

# Anlage KAP

In [ ]:
optStr, stockStr, cfdStr, futStr = "", "", "", ""
if PL_Trades.index.str.contains("Option").sum() > 0:
    optStr = PL_Trades.index[PL_Trades.index.str.contains("Option")][0]
if PL_Trades.index.str.contains("Stock").sum() > 0:
    stockStr = PL_Trades.index[PL_Trades.index.str.contains("Stock")][0]
if PL_Trades.index.str.contains("CFD").sum() > 0:
    cfdStr = PL_Trades.index[PL_Trades.index.str.contains("CFD")][0]
if PL_Trades.index.str.contains("Futures").sum() > 0:
    futStr = PL_Trades.index[PL_Trades.index.str.contains("Futures")][0]

In [ ]:
includeCfd = False
lst_kap = []

# Line 18 German capital income, Interest + option premium
# according to pwc report without dividend ger
dividendGer = Dividends[Dividends.index.get_level_values(2) == 'DE'].sum()["Amount [€]"]
interest = Interest.sum()["Amount [€]"]["Received"]
#TODO: only german, instead of current only eur
optPremEur = PL_TradesDet["P [€]"]["EUR"][optStr] \
    if optStr in PL_TradesDet["P [€]"]["EUR"].index \
    else 0

if OptionsDet.empty:
    optPremEur = 0
else:
    try:
        # hard coded, german stocks selected
        gerStock = "FRE|SAP"
        gerStockMask = OptionsDet.Symbol.str.contains(gerStock)
        optPremEur = OptionsDet[gerStockMask]["Pnl"].sum()
    except:
        # if first 3 options trades are german
        optPremEur = OptionsDet[:3]["Pnl"].sum()
        
capIncomeGer = interest + optPremEur

# deprecated
mask = (PL_TradesDet["P [€]"].index.get_level_values(0) != "EUR") \
    & (PL_TradesDet["P [€]"].index.get_level_values(1) == optStr)
optPrem = PL_TradesDet["P [€]"][mask].sum()
# new
optPrem = Options["P [€]"].sum() - optPremEur if not Options.empty else 0

# Line 20 Stock Profit
stockProfit = PL_Trades["P [€]"][stockStr] if stockStr in PL_Trades["P [€]"].index else 0

# Line 22 non Stock loss (future loss + option loss)
# nonStockLoss = PL_Trades["L [€]"][(PL_Trades["L [€]"].index != stockStr) & (PL_Trades["L [€]"].index != cfdStr)].sum()
nonStockLoss = PL_Trades["L [€]"][futStr] if futStr in PL_Trades["L [€]"].index else 0
nonStockLoss = nonStockLoss + Options["L [€]"].sum() if not Options.empty else 0 

# Line 23 Stock loss
stockLoss = PL_Trades["L [€]"][stockStr] if stockStr in PL_Trades["L [€]"].index else 0

# Line 19 Foreign capital income
# stock profit + other loss + stock loss + option premium + dividend - line18
capIncome = stockProfit + nonStockLoss + stockLoss + optPrem + Dividends.sum()["Amount [€]"] - dividendGer

# cfd
if includeCfd:
    cfdLoss = PL_TradesDet[PL_TradesDet.index.get_level_values(1) == cfdStr]["L [€]"].sum()
    cfdProfit = PL_TradesDet[PL_TradesDet.index.get_level_values(1) == cfdStr]["P [€]"].sum()
    cfdDividend = Dividends[Dividends.index.get_level_values(1) == cfdStr]["Amount [€]"].sum()
    
    nonStockLoss = nonStockLoss + cfdLoss
    capIncome = capIncome + cfdProfit + cfdDividend

lst_kap.append(["Line 18", capIncomeGer])
lst_kap.append(["Line 19", capIncome])
lst_kap.append(["Line 20", stockProfit])
lst_kap.append(["Line 22", nonStockLoss])
lst_kap.append(["Line 23", stockLoss])
KAP = pd.DataFrame(lst_kap, columns=["Name","Value"])

# Final adjustments

In [ ]:
WithholdingTax = addSumRow(WithholdingTax)
Dividends = addSumRow(Dividends)
Interest = addSumRow(Interest)
PL_Trades = addSumRow(PL_Trades)
PL_TradesDet = addSumRow(PL_TradesDet)
OptionsDet = addSumRow(OptionsDet) if not OptionsDet.empty else OptionsDet
FuturesDet = addSumRow(FuturesDet) if not FuturesDet.empty else FuturesDet
CfdDet = addSumRow(CfdDet) if not CfdDet.empty else CfdDet

# Results

In [ ]:
KAP

In [ ]:
Interest

In [ ]:
WithholdingTax if not WithholdingTax.empty else "No Withholding Tax"

In [ ]:
Dividends

In [ ]:
PL_Trades

In [ ]:
PL_TradesDet

# Detailed Results

In [ ]:
display("Options", Options, OptionsDet)

In [ ]:
display("Futures", Futures, FuturesDet)

In [ ]:
display("CFD", Cfd, CfdDet)

---

# P/L Forex

## Not fully tested

### Only applies if every position is closed and opened during the year (i.e. Options)

Use > Realized & Unrealized Performance Summary > Forex

In [ ]:
assets  = df_trades.Asset.unique()
results = []
for asset in assets:
    try:
        df_asstes = df_trades[df_trades.Asset == asset].copy()
        df_asstes["Basis"] = df_asstes["Basis"].astype(float)
        df_asstes["Basis [€]"] = df_asstes.apply(lambda row: c.convert(
            row["Basis"] , row.Currency, date=row["Date/Time"]), axis=1)
        currencies = df_asstes.Currency.unique()
        
        for curr in currencies:
            df_curr = df_asstes[df_asstes.Currency==curr].copy()
            open_position = df_curr["Basis"].round(2).sum()
            pl_forex = df_curr["Basis [€]"].round(2).sum()
            avg_rate = df_asstes.apply(lambda row: c.convert(
            1 , row.Currency, date=row["Date/Time"]), axis=1).mean()
            results.append([curr, avg_rate, pl_forex, open_position, asset])
            #results.append([curr, pl_forex, open_position, asset])
            
    except Exception as e:
        print(f"Failed for {asset} with error: {e}")
df_forex = pd.DataFrame(results).T
df_forex, df_forex.columns = df_forex.iloc[1:] , df_forex.iloc[0]
df_forex = df_forex.T
df_forex.columns = ["Average Rate", "PL_Forex [€]", "Open Position", "Asset"]
#df_forex.columns = ["PL_Forex [€]", "Open Position", "Asset"]
df_forex.index.name = None
df_forex["PL_Forex_ADJ [€]"] = df_forex["PL_Forex [€]"] - df_forex["Open Position"]
df_forex.loc[:,:] = df_forex.loc[:,:].apply(pd.to_numeric, errors = 'ignore')
df_forex.loc['Column_Total'] = df_forex.sum(numeric_only=True, axis=0)
PL_Forex = df_forex
PL_Forex

# Code Cementary

In [ ]:
try:
    df_options["TPrice"] = df_options["T. Price"].astype(float) * 100
    df_options["TPrice"] = df_options["TPrice"] * -df_options["Quantity"].astype(int)
    df_options["TPrice"] = df_options["TPrice"] + df_options["Comm/Fee"].astype(float) 
    df_options["Pnl"] = df_options.apply(lambda row: c.convert(
                row["TPrice"] , row.Currency, date=row["Date/Time"]), axis=1)
    df_options["cummulative"] = df_options["Pnl"].cumsum()

    df_options["Open"] = df_options.Code.str.match(r'(C)(?!\w)') # regex should also include (O) not prepended with word char
    df_options["Open"] = df_options["Open"].cumsum().shift().fillna(0)

    s1 = df_options.groupby("Open").last()["Symbol"]
    s2 = df_options.groupby("Open").sum()["Pnl"]
except:
    s1 = pd.Series([0], name="Symbol")
    s2 = pd.Series([0], name="Pnl")

In [ ]:
OptionsDet = pd.concat([s1, s2], axis=1)

Options = pd.DataFrame()
try:
    Options["P [€]"] = [OptionsDet["Pnl"].apply(lambda row: row if row > 0 else 0).sum()]
    Options["L [€]"] = [OptionsDet["Pnl"].apply(lambda row: row if row < 0 else 0).sum()]
except:
    pass